# UAC Care Pipeline — KPI Visualization Notebook

**5 KPIs × 4–5 charts each** using Matplotlib & Seaborn.

KPIs: Transfer Efficiency · Discharge Effectiveness · Pipeline Throughput · Backlog Accumulation · Outcome Stability

---

## ⚙️ Setup — Data Load & KPI Calculation

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# ── Load & prepare data ──────────────────────────────────────────────────────
df = pd.read_csv('uac_data.csv')
df['Date'] = pd.to_datetime(df['Date'])
df.rename(columns={
    'Children apprehended and placed in CBP custody':'cbp_in',
    'Children in CBP custody':'cbp_cus',
    'Children transferred out of CBP custody':'cbp_tr',
    'Children in HHS Care':'hhs_care',
    'Children discharged from HHS Care':'hhs_dc'
}, inplace=True)

# ── KPI calculations ─────────────────────────────────────────────────────────
df['transfer_eff']   = (df['cbp_tr']   / df['cbp_cus'] ).replace([np.inf,-np.inf],np.nan)
df['discharge_eff']  = (df['hhs_dc']   / df['hhs_care']).replace([np.inf,-np.inf],np.nan)
df['throughput']     = (df['hhs_dc']   / df['cbp_in']  ).replace([np.inf,-np.inf],np.nan)
df['net_change']     = df['cbp_in']    - df['hhs_dc']
df['cum_backlog']    = df['net_change'].cumsum()
df['sys_load']       = df['cbp_cus']   + df['hhs_care']
df['cbp_net']        = df['cbp_in']    - df['cbp_tr']
df['hhs_net']        = df['cbp_tr']    - df['hhs_dc']

# ── Rolling / temporal features ──────────────────────────────────────────────
for col in ['transfer_eff','discharge_eff','throughput']:
    df[col+'_30'] = df[col].rolling(30).mean()
df['dc_roll_std'] = df['discharge_eff'].rolling(30).std()
df['month']     = df['Date'].dt.month
df['dow']       = df['Date'].dt.dayofweek
df['is_wknd']   = df['dow'].isin([5,6]).astype(int)
df['year']      = df['Date'].dt.year
df['quarter']   = df['Date'].dt.to_period('Q').astype(str)
df['mon_name']  = df['Date'].dt.strftime('%b')
df['de_diff']   = df['discharge_eff'].diff()

MN = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

# ── Colour palette ────────────────────────────────────────────────────────────
C = dict(good='#27AE60', warn='#E67E22', bad='#C0392B',
         blue='#2980B9', purple='#8E44AD', dark='#2C3E50',
         lg='#A9DFBF', lb='#FADBD8', lw='#FDEBD0', bg='#F8F9FA')

plt.rcParams.update({'axes.facecolor':C['bg'],'figure.facecolor':'white',
                     'axes.spines.top':False,'axes.spines.right':False,
                     'axes.grid':True,'grid.alpha':0.3,'grid.linestyle':'--'})
print("Setup complete — data shape:", df.shape)

---
## 📊 KPI #1 — Transfer Efficiency Ratio
> *Measures CBP → HHS speed | Formula: Transfers ÷ CBP Custody | Target >80%*

**Charts:** ① Time-Series Trend  ② Monthly Avg  ③ Weekday vs Weekend  ④ Performance Zone Donut  ⑤ Distribution + KDE

In [ ]:
# ═══════════════════════════════════════════════════════════════════
#  KPI 1 ▸ TRANSFER EFFICIENCY RATIO
#  Formula: Transfers ÷ CBP Custody  |  Target: >80%
#  Story: How fast does CBP hand children off to HHS?
# ═══════════════════════════════════════════════════════════════════

te_mean = df['transfer_eff'].mean()
te_med  = df['transfer_eff'].median()
te_std  = df['transfer_eff'].std()
days_ok = (df['transfer_eff'] >= 0.8).sum()
days_fail=(df['transfer_eff'] <  0.8).sum()

fig = plt.figure(figsize=(22,20))
fig.patch.set_facecolor('white')
fig.text(0.5,0.98,'KPI #1 — Transfer Efficiency Ratio',
         ha='center',fontsize=24,fontweight='bold',color=C['dark'])
fig.text(0.5,0.955,
         f'Measures CBP → HHS speed  |  Formula: Transfers ÷ CBP Custody  |  Target >80%'
         f'  |  Mean: {te_mean:.1%}  |  Status: {"✓ PASS" if te_mean>=0.8 else "✗ BELOW TARGET"}',
         ha='center',fontsize=12,color='gray')

gs = gridspec.GridSpec(3,3,figure=fig,top=0.94,bottom=0.05,
                       hspace=0.50,wspace=0.38)

# ─── CHART 1: Time-series trend with threshold bands ─────────────────────────
ax1 = fig.add_subplot(gs[0,:])
ax1.fill_between(df['Date'],0.8,1.08,color=C['lg'],alpha=0.45,label='Target Zone ≥80%')
ax1.fill_between(df['Date'],0,0.8,  color=C['lb'],alpha=0.45,label='Below Target <80%')
ax1.plot(df['Date'],df['transfer_eff'],   lw=1.1,color=C['blue'],alpha=0.40,label='Daily')
ax1.plot(df['Date'],df['transfer_eff_30'],lw=2.8,color=C['dark'],label='30-Day Rolling Mean')
ax1.axhline(0.8,   color=C['bad'], lw=2.0,ls=':',label='Policy Target 80%')
ax1.axhline(te_mean,color=C['warn'],lw=2.0,ls='--',label=f'Actual Mean {te_mean:.1%}')
ax1.set(title='① Time-Series Trend with Performance Bands',
        ylabel='Transfer Efficiency Ratio',xlabel='Date',ylim=(0,1.1))
ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda y,_:f'{y:.0%}'))
ax1.legend(ncol=3,fontsize=9,loc='lower left',framealpha=0.9)
ax1.text(0.01,0.96,f'Mean {te_mean:.1%} | Median {te_med:.1%} | σ {te_std:.2%}',
         transform=ax1.transAxes,fontsize=10,va='top',
         bbox=dict(boxstyle='round',facecolor=C['lw'],alpha=0.9))

# ─── CHART 2: Monthly average bar ────────────────────────────────────────────
ax2 = fig.add_subplot(gs[1,0])
mon_te = df.groupby('month')['transfer_eff'].mean()
bc2 = [C['good'] if v>=0.8 else C['bad'] for v in mon_te.values]
bars=ax2.bar(mon_te.index,mon_te.values,color=bc2,edgecolor='white',lw=1.5,width=0.75)
ax2.axhline(0.8,color='red',lw=2,ls='--',label='Target')
ax2.set(title='② Monthly Average',xlabel='Month',
        ylabel='Avg Efficiency',xticks=range(1,13))
ax2.set_xticklabels(MN,fontsize=8)
ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda y,_:f'{y:.0%}'))
for b,v in zip(bars,mon_te.values):
    ax2.text(b.get_x()+b.get_width()/2,v+0.005,f'{v:.0%}',
             ha='center',va='bottom',fontsize=7.5,fontweight='bold')
ax2.legend(fontsize=9)

# ─── CHART 3: Weekday vs Weekend grouped bars ─────────────────────────────────
ax3 = fig.add_subplot(gs[1,1])
wk = df.groupby('is_wknd')['transfer_eff'].agg(['mean','std'])
labels=['Weekday','Weekend']
x=np.arange(2)
bars3=ax3.bar(x,wk['mean'].values,yerr=wk['std'].values,
              color=[C['blue'],C['warn']],edgecolor='white',
              width=0.5,capsize=6,error_kw=dict(lw=2))
ax3.axhline(0.8,color='red',lw=2,ls='--')
ax3.set(title='③ Weekday vs Weekend',ylabel='Avg Efficiency',
        xticks=x,xticklabels=labels)
ax3.yaxis.set_major_formatter(plt.FuncFormatter(lambda y,_:f'{y:.0%}'))
diff_pct=(wk['mean'].iloc[1]/wk['mean'].iloc[0]-1)*100
ax3.text(1,wk['mean'].iloc[1]+wk['std'].iloc[1]+0.02,
         f'{diff_pct:+.1f}%\nvs wkday',ha='center',fontsize=9,
         fontweight='bold',color=C['bad'] if diff_pct<0 else C['good'])
for b,v in zip(bars3,wk['mean'].values):
    ax3.text(b.get_x()+b.get_width()/2,v-0.03,f'{v:.1%}',
             ha='center',fontsize=10,fontweight='bold',color='white')

# ─── CHART 4: Performance zone donut ─────────────────────────────────────────
ax4 = fig.add_subplot(gs[1,2])
zones={'Excellent\n≥90%':(df['transfer_eff']>=0.9).sum(),
       'Good\n80–90%':((df['transfer_eff']>=0.8)&(df['transfer_eff']<0.9)).sum(),
       'Below\n50–80%':((df['transfer_eff']>=0.5)&(df['transfer_eff']<0.8)).sum(),
       'Critical\n<50%':(df['transfer_eff']<0.5).sum()}
zc=[C['good'],C['blue'],C['warn'],C['bad']]
wedges,texts,autos=ax4.pie(
    zones.values(),labels=zones.keys(),colors=zc,
    autopct='%1.0f%%',startangle=90,pctdistance=0.75,
    wedgeprops=dict(edgecolor='white',lw=2.5,width=0.55))
for a in autos: a.set_fontsize(9);a.set_fontweight('bold')
ax4.set_title('④ Days by Performance Zone')
ax4.text(0,0,f'{te_mean:.0%}\nMean',ha='center',va='center',
         fontsize=13,fontweight='bold',color=C['dark'])

# ─── CHART 5: Distribution histogram with KDE ────────────────────────────────
ax5 = fig.add_subplot(gs[2,:])
sns.histplot(df['transfer_eff'].dropna(),bins=70,ax=ax5,
             color=C['blue'],alpha=0.65,edgecolor='white',stat='count')
ax5_k=ax5.twinx()
sns.kdeplot(df['transfer_eff'].dropna(),ax=ax5_k,color=C['dark'],lw=2.5,label='KDE')
ax5_k.set_ylabel('Density',fontsize=11)
ax5.axvline(te_mean, color=C['warn'],  lw=2.5,ls='--',label=f'Mean {te_mean:.1%}')
ax5.axvline(te_med,  color=C['good'],  lw=2.5,ls='-.',label=f'Median {te_med:.1%}')
ax5.axvline(0.8,     color=C['bad'],   lw=2.5,ls=':',label='Target 80%')
ax5.set(title='⑤ Full Distribution of Transfer Efficiency — How Often Each Level Occurs',
        xlabel='Transfer Efficiency Ratio',ylabel='Number of Days')
ax5.xaxis.set_major_formatter(plt.FuncFormatter(lambda x,_:f'{x:.0%}'))
lines1,labs1=ax5.get_legend_handles_labels()
lines2,labs2=ax5_k.get_legend_handles_labels()
ax5.legend(lines1+lines2,labs1+labs2,fontsize=10,loc='upper left')
ax5.text(0.98,0.92,
         f'Days ≥Target: {days_ok} ({days_ok/len(df)*100:.0f}%)\n'
         f'Days <Target: {days_fail} ({days_fail/len(df)*100:.0f}%)',
         transform=ax5.transAxes,ha='right',va='top',fontsize=11,
         bbox=dict(boxstyle='round',facecolor=C['lb'],alpha=0.9))

plt.savefig('KPI1_Transfer_Efficiency.png',dpi=150,bbox_inches='tight')
plt.show()
print(f"KPI1 done | Mean={te_mean:.1%} | Days below target={days_fail}")

---
## 📊 KPI #2 — Discharge Effectiveness Index
> *Placement success | Formula: Discharges ÷ HHS Care | Target >5% daily*

**Charts:** ① Control-Band Time-Series  ② Quarterly Boxplot  ③ Length-of-Stay Bars  ④ Rolling Variability  ⑤ KPI Scorecard

In [ ]:
# ═══════════════════════════════════════════════════════════════════
#  KPI 2 ▸ DISCHARGE EFFECTIVENESS INDEX
#  Formula: Discharges ÷ HHS Care  |  Target: >5% daily
#  Story: How fast are children placed with sponsors?
# ═══════════════════════════════════════════════════════════════════

de_mean = df['discharge_eff'].mean()
de_med  = df['discharge_eff'].median()
de_std  = df['discharge_eff'].std()
cv      = de_std/de_mean
avg_los = 1/de_mean
drops   = df[df['de_diff'] < -0.05]

fig = plt.figure(figsize=(22,20))
fig.patch.set_facecolor('white')
fig.text(0.5,0.98,'KPI #2 — Discharge Effectiveness Index',
         ha='center',fontsize=24,fontweight='bold',color=C['dark'])
fig.text(0.5,0.955,
         f'Placement success  |  Formula: Discharges ÷ HHS Care  |  Target >5% daily'
         f'  |  Mean: {de_mean:.2%}  |  Avg Stay: {avg_los:.0f} days',
         ha='center',fontsize=12,color='gray')

gs = gridspec.GridSpec(3,3,figure=fig,top=0.94,bottom=0.05,
                       hspace=0.50,wspace=0.38)

# ─── CHART 1: Time-series + control bands + drop markers ─────────────────────
ax1 = fig.add_subplot(gs[0,:])
ax1.fill_between(df['Date'],de_mean-de_std,de_mean+de_std,
                 color=C['lg'],alpha=0.35,label='Normal Range ±1σ')
ax1.plot(df['Date'],df['discharge_eff'],   lw=1.0,color=C['purple'],alpha=0.40,label='Daily')
ax1.plot(df['Date'],df['discharge_eff_30'],lw=2.8,color=C['purple'],label='30-Day Trend')
ax1.axhline(de_mean,color='blue',lw=2.0,ls='--',label=f'Mean {de_mean:.2%}')
ax1.axhline(0.05,   color=C['bad'],lw=2.0,ls=':',label='Min Target 5%')
if len(drops):
    ax1.scatter(drops['Date'],drops['discharge_eff'],
                color='red',s=90,zorder=5,marker='v',
                label=f'Sudden Drops ({len(drops)})')
ax1.set(title='① Time-Series: Placement Rate with Control Bands + Sudden Drops',
        ylabel='Discharge Effectiveness',xlabel='Date')
ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda y,_:f'{y:.1%}'))
ax1.legend(ncol=3,fontsize=9,loc='upper right',framealpha=0.9)
ax1.text(0.01,0.96,
         f'Mean {de_mean:.2%} | CV={cv:.2f} | Avg Stay {avg_los:.0f}d | Drops: {len(drops)}',
         transform=ax1.transAxes,fontsize=10,va='top',
         bbox=dict(boxstyle='round',facecolor=C['lw'],alpha=0.9))

# ─── CHART 2: Quarterly boxplot ───────────────────────────────────────────────
ax2 = fig.add_subplot(gs[1,:2])
q_grps=[g['discharge_eff'].dropna().values for _,g in df.groupby('quarter')]
q_labs=[q for q,_ in df.groupby('quarter')]
bp=ax2.boxplot(q_grps,labels=q_labs,patch_artist=True,
               showmeans=True,meanline=True,
               whiskerprops=dict(lw=1.5),
               medianprops=dict(lw=2.5,color='white'))
for patch,lbl in zip(bp['boxes'],q_labs):
    qm=df[df['quarter']==lbl]['discharge_eff'].mean()
    patch.set_facecolor(C['good'] if qm>=0.05 else C['bad'])
    patch.set_alpha(0.75)
ax2.axhline(0.05,color='red',lw=2,ls='--',label='Target 5%')
ax2.axhline(de_mean,color='blue',lw=1.8,ls='--',alpha=0.7,label=f'Mean {de_mean:.2%}')
ax2.set(title='② Quarterly Distribution — Consistency Check Across Time',
        ylabel='Discharge Effectiveness')
ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda y,_:f'{y:.1%}'))
ax2.legend(fontsize=9); plt.setp(ax2.get_xticklabels(),rotation=45,ha='right',fontsize=8)

# ─── CHART 3: Length-of-stay bar ─────────────────────────────────────────────
ax3 = fig.add_subplot(gs[1,2])
los={'<13d\n>8%':(df['discharge_eff']>0.08).sum(),
     '13–20d\n5–8%':((df['discharge_eff']>=0.05)&(df['discharge_eff']<=0.08)).sum(),
     '21–33d\n3–5%':((df['discharge_eff']>=0.03)&(df['discharge_eff']<0.05)).sum(),
     '>33d\n<3%':(df['discharge_eff']<0.03).sum()}
bc3=[C['good'],C['blue'],C['warn'],C['bad']]
bars3=ax3.bar(range(4),list(los.values()),color=bc3,edgecolor='white',lw=1.5)
ax3.set(title='③ Days by Length-\nof-Stay Category',xticks=range(4),ylabel='Days')
ax3.set_xticklabels(list(los.keys()),fontsize=8)
for b,v in zip(bars3,los.values()):
    if v: ax3.text(b.get_x()+b.get_width()/2,v+2,f'{v}\n({v/len(df)*100:.0f}%)',
                   ha='center',va='bottom',fontsize=8,fontweight='bold')

# ─── CHART 4: Rolling variability (stability) ─────────────────────────────────
ax4 = fig.add_subplot(gs[2,:2])
ax4.plot(df['Date'],df['dc_roll_std'],lw=2.5,color=C['purple'],label='30-Day Rolling σ')
ax4.fill_between(df['Date'],0,df['dc_roll_std'],alpha=0.20,color=C['purple'])
ax4.axhline(de_std,color='blue',lw=2,ls='--',label=f'Overall σ={de_std:.3f}')
ax4.set(title='④ Rolling Variability (Outcome Stability) — Lower = More Consistent Placements',
        xlabel='Date',ylabel='Rolling Std Dev')
ax4.legend(fontsize=10)

# ─── CHART 5: KPI scorecard text panel ───────────────────────────────────────
ax5 = fig.add_subplot(gs[2,2])
ax5.axis('off')
rating='★★★★★' if de_mean>=0.08 else ('★★★★☆' if de_mean>=0.05 else '★★★☆☆')
status='✓ PASS' if de_mean>=0.05 else '✗ BELOW TARGET'
sc_items=[
    ('Mean Effectiveness',f'{de_mean:.2%}'),
    ('Median',f'{de_med:.2%}'),
    ('Avg Length of Stay',f'{avg_los:.0f} days'),
    ('Std Dev (σ)',f'{de_std:.4f}'),
    ('CV (stability)',f'{cv:.3f}'),
    ('Days < Target',f'{(df["discharge_eff"]<0.05).sum()}'),
    ('Sudden Drops',f'{len(drops)}'),
    ('Rating',rating),
    ('Status',status),
]
ax5.text(0.5,1.02,'KPI #2 Scorecard',ha='center',va='top',
         fontsize=13,fontweight='bold',transform=ax5.transAxes)
yp=0.90
for lbl,val in sc_items:
    col=C['good'] if '✓' in val else (C['bad'] if '✗' in val else C['dark'])
    ax5.text(0.03,yp,lbl+':',fontsize=10,va='top',transform=ax5.transAxes,color='gray')
    ax5.text(0.97,yp,val,fontsize=10,va='top',ha='right',
             transform=ax5.transAxes,fontweight='bold',color=col)
    ax5.add_patch(mpatches.FancyBboxPatch((0,yp-0.005),1,0.085,
        boxstyle='round,pad=0.01',fc='#F2F3F4',ec='none',
        transform=ax5.transAxes,zorder=0))
    yp-=0.098

plt.savefig('KPI2_Discharge_Effectiveness.png',dpi=150,bbox_inches='tight')
plt.show()
print(f"KPI2 done | Mean={de_mean:.2%} | AvgLOS={avg_los:.0f}d | CV={cv:.3f}")

---
## 📊 KPI #3 — Pipeline Throughput
> *Overall system movement | Formula: Total Exits ÷ Entries | Target ~100%*

**Charts:** ① Cumulative Area Flow  ② Daily Throughput Ratio  ③ Balance Category Donut  ④ Monthly Bar  ⑤ Year-over-Year Line

In [ ]:
# ═══════════════════════════════════════════════════════════════════
#  KPI 3 ▸ PIPELINE THROUGHPUT
#  Formula: Total Exits ÷ Total Entries  |  Target: ~100%
#  Story: Is the system clearing children as fast as they arrive?
# ═══════════════════════════════════════════════════════════════════

cum_in  = df['cbp_in'].cumsum()
cum_out = df['hhs_dc'].cumsum()
cum_gap = cum_in - cum_out
tp_mean = df['throughput'].mean()
tp_tot  = cum_out.iloc[-1]/cum_in.iloc[-1]
final_gap=cum_gap.iloc[-1]

fig = plt.figure(figsize=(22,20))
fig.patch.set_facecolor('white')
fig.text(0.5,0.98,'KPI #3 — Pipeline Throughput',
         ha='center',fontsize=24,fontweight='bold',color=C['dark'])
fig.text(0.5,0.955,
         f'Overall system movement  |  Formula: Exits ÷ Entries  |  Target ~100%'
         f'  |  Overall Throughput: {tp_tot:.1%}  |  Gap: {final_gap:,.0f} children',
         ha='center',fontsize=12,color='gray')

gs = gridspec.GridSpec(3,3,figure=fig,top=0.94,bottom=0.05,
                       hspace=0.50,wspace=0.38)

# ─── CHART 1: Cumulative flow area chart ─────────────────────────────────────
ax1 = fig.add_subplot(gs[0,:])
ax1.plot(df['Date'],cum_in, lw=3.2,color=C['bad'], label=f'Cumulative Entries  {cum_in.iloc[-1]:,.0f}')
ax1.plot(df['Date'],cum_out,lw=3.2,color=C['good'],label=f'Cumulative Exits  {cum_out.iloc[-1]:,.0f}')
ax1.fill_between(df['Date'],cum_in,cum_out,alpha=0.30,color=C['warn'],
                 label=f'Accumulation Gap  {final_gap:,.0f}')
ax1.annotate(f'GAP\n{final_gap:,.0f}',
             xy=(df['Date'].iloc[-1],cum_in.iloc[-1]),
             xytext=(-110,-55),textcoords='offset points',
             bbox=dict(boxstyle='round,pad=0.6',facecolor='yellow',
                       alpha=0.95,edgecolor='red',lw=2.5),
             arrowprops=dict(arrowstyle='->',lw=2.5,color='red'),
             fontsize=13,fontweight='bold')
ax1.set(title='① Cumulative Flow Diagram — Total Entries vs Total Exits (Gap = Accumulated Cases)',
        ylabel='Cumulative Count',xlabel='Date')
ax1.legend(fontsize=11,loc='upper left',framealpha=0.9)
ax1.text(0.01,0.96,f'Overall throughput {tp_tot:.1%} | Mean daily {tp_mean:.1%}',
         transform=ax1.transAxes,fontsize=10,va='top',
         bbox=dict(boxstyle='round',facecolor=C['lw'],alpha=0.9))

# ─── CHART 2: Daily throughput ratio line ─────────────────────────────────────
ax2 = fig.add_subplot(gs[1,:2])
ax2.fill_between(df['Date'],0.9,1.1,color=C['lg'],alpha=0.40,label='Balanced Zone (90–110%)')
ax2.fill_between(df['Date'],0,  0.9,color=C['lb'],alpha=0.35,label='Accumulation Zone <90%')
ax2.plot(df['Date'],df['throughput'],   lw=1.0,color=C['warn'],alpha=0.45,label='Daily Ratio')
ax2.plot(df['Date'],df['throughput_30'],lw=2.8,color=C['dark'],label='30-Day Trend')
ax2.axhline(1.0,    color=C['good'],lw=2.5,ls='-', label='Balanced (100%)')
ax2.axhline(tp_mean,color=C['bad'], lw=2.0,ls='--',label=f'Mean {tp_mean:.1%}')
ax2.set(title='② Daily Throughput Ratio — Are Exits Keeping Pace with Entries?',
        ylabel='Ratio (Exits ÷ Entries)',xlabel='Date')
ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda y,_:f'{y:.0%}'))
ax2.legend(fontsize=9,ncol=3,loc='lower left')

# ─── CHART 3: Throughput category donut ──────────────────────────────────────
ax3 = fig.add_subplot(gs[1,2])
cats={'Clearing\n>110%':(df['throughput']>1.1).sum(),
      'Balanced\n90–110%':((df['throughput']>=0.9)&(df['throughput']<=1.1)).sum(),
      'Mild Acc\n70–90%':((df['throughput']>=0.7)&(df['throughput']<0.9)).sum(),
      'Severe\n<70%':(df['throughput']<0.7).sum()}
pc=[C['good'],C['blue'],C['warn'],C['bad']]
w,t,a=ax3.pie(cats.values(),labels=cats.keys(),colors=pc,
              autopct='%1.0f%%',startangle=90,pctdistance=0.75,
              wedgeprops=dict(edgecolor='white',lw=2.5,width=0.55))
for aa in a: aa.set_fontsize(9);aa.set_fontweight('bold')
ax3.set_title('③ Days by System\nBalance Category')
ax3.text(0,0,f'{tp_tot:.0%}',ha='center',va='center',
         fontsize=16,fontweight='bold',color=C['dark'])

# ─── CHART 4: Monthly throughput bar ─────────────────────────────────────────
ax4 = fig.add_subplot(gs[2,:2])
mon_tp=df.groupby('month').apply(lambda x: x['hhs_dc'].sum()/x['cbp_in'].sum())
bc4=[C['good'] if v>=0.9 else (C['warn'] if v>=0.7 else C['bad']) for v in mon_tp.values]
bars4=ax4.bar(mon_tp.index,mon_tp.values,color=bc4,edgecolor='white',lw=1.5,width=0.75)
ax4.axhline(1.0,    color='green',lw=2.5,ls='-', label='Balanced 100%')
ax4.axhline(tp_tot, color='blue', lw=2.0,ls='--',label=f'Overall {tp_tot:.1%}')
ax4.set(title='④ Monthly Throughput — Which Months Accumulate Most?',
        xlabel='Month',ylabel='Throughput Ratio',xticks=range(1,13))
ax4.set_xticklabels(MN)
ax4.yaxis.set_major_formatter(plt.FuncFormatter(lambda y,_:f'{y:.0%}'))
ax4.legend(fontsize=10)
for b,v in zip(bars4,mon_tp.values):
    ax4.text(b.get_x()+b.get_width()/2,v+0.01,f'{v:.0%}',
             ha='center',va='bottom',fontsize=8.5,fontweight='bold')

# ─── CHART 5: Year-over-Year line ─────────────────────────────────────────────
ax5 = fig.add_subplot(gs[2,2])
pal=['#2980B9','#E67E22']
for (yr,grp),col in zip(df.groupby('year'),pal):
    mtp=grp.groupby('month').apply(lambda x: x['hhs_dc'].sum()/x['cbp_in'].sum())
    ax5.plot(mtp.index,mtp.values,marker='o',lw=2.5,
             markersize=7,color=col,label=str(yr))
ax5.axhline(1.0,color='green',lw=2,ls='--',alpha=0.7,label='Balanced')
ax5.set(title='⑤ Year-over-Year\nComparison',
        xlabel='Month',ylabel='Throughput')
ax5.set_xticks(range(1,13)); ax5.set_xticklabels(MN,fontsize=8)
ax5.yaxis.set_major_formatter(plt.FuncFormatter(lambda y,_:f'{y:.0%}'))
ax5.legend(fontsize=10)

plt.savefig('KPI3_Pipeline_Throughput.png',dpi=150,bbox_inches='tight')
plt.show()
print(f"KPI3 done | Throughput={tp_tot:.1%} | Gap={final_gap:,.0f}")

---
## 📊 KPI #4 — Backlog Accumulation Rate
> *Delay severity | Formula: Daily Entries − Exits | Target: 0/day*

**Charts:** ① Waterfall + Cumulative  ② Stage Breakdown  ③ Streak Duration  ④ System Load Area  ⑤ Monthly Heatmap

In [ ]:
# ═══════════════════════════════════════════════════════════════════
#  KPI 4 ▸ BACKLOG ACCUMULATION RATE
#  Formula: Daily Entries − Daily Exits  |  Target: 0/day
#  Story: Where and when do children get stuck?
# ═══════════════════════════════════════════════════════════════════

avg_acc  = df['net_change'].mean()
final_bk = df['cum_backlog'].iloc[-1]
days_acc = (df['net_change']>0).sum()
days_clr = (df['net_change']<=0).sum()
cbp_tot  = df['cbp_net'].sum()
hhs_tot  = df['hhs_net'].sum()
bottleneck='HHS Stage' if abs(hhs_tot)>abs(cbp_tot) else 'CBP Stage'

# Streak analysis
df['acc_flag']=(df['net_change']>0).astype(int)
df['streak_g']=(df['acc_flag']!=df['acc_flag'].shift()).cumsum()
streaks=df[df['acc_flag']==1].groupby('streak_g').size()
max_streak=int(streaks.max()) if len(streaks) else 0

fig = plt.figure(figsize=(22,20))
fig.patch.set_facecolor('white')
fig.text(0.5,0.98,'KPI #4 — Backlog Accumulation Rate',
         ha='center',fontsize=24,fontweight='bold',color=C['dark'])
fig.text(0.5,0.955,
         f'Delay severity  |  Formula: Daily Entries − Exits  |  Target 0/day'
         f'  |  Avg {avg_acc:+.1f}/day  |  Bottleneck: {bottleneck}  |  Max Streak: {max_streak}d',
         ha='center',fontsize=12,color='gray')

gs = gridspec.GridSpec(3,3,figure=fig,top=0.94,bottom=0.05,
                       hspace=0.50,wspace=0.38)

# ─── CHART 1: Waterfall bars + cumulative line overlay ───────────────────────
ax1 = fig.add_subplot(gs[0,:])
bc1=[C['bad'] if v>0 else C['good'] for v in df['net_change']]
ax1.bar(df['Date'],df['net_change'],color=bc1,alpha=0.60,width=1,label='Daily Net Change')
ax1r=ax1.twinx()
ax1r.plot(df['Date'],df['cum_backlog'],lw=3.2,color=C['dark'],label='Cumulative Backlog',zorder=5)
ax1r.fill_between(df['Date'],df['cum_backlog'],0,
                  where=(df['cum_backlog']>0), alpha=0.18,color='red')
ax1r.fill_between(df['Date'],df['cum_backlog'],0,
                  where=(df['cum_backlog']<=0),alpha=0.18,color='green')
ax1r.axhline(0,color='black',lw=1.5)
ax1r.annotate(f'Final\n{final_bk:+,.0f}',
              xy=(df['Date'].iloc[-1],final_bk),
              xytext=(-90,30),textcoords='offset points',
              bbox=dict(boxstyle='round,pad=0.6',facecolor='yellow',
                        alpha=0.95,edgecolor='red',lw=2.5),
              arrowprops=dict(arrowstyle='->',lw=2,color='red'),
              fontsize=12,fontweight='bold',zorder=6)
ax1.set(title='① Waterfall Chart — Daily Net Change (Bars) + Cumulative Backlog (Line)',
        ylabel='Daily Net Change',xlabel='Date')
ax1r.set_ylabel('Cumulative Backlog',color=C['dark'])
l1,lb1=ax1.get_legend_handles_labels()
l2,lb2=ax1r.get_legend_handles_labels()
ax1.legend(l1+l2,lb1+lb2,fontsize=9,loc='upper left')
ax1.text(0.01,0.96,f'Avg {avg_acc:+.1f}/day | Acc days {days_acc} | Clr days {days_clr}',
         transform=ax1.transAxes,fontsize=10,va='top',
         bbox=dict(boxstyle='round',facecolor=C['lw'],alpha=0.9))

# ─── CHART 2: Stage breakdown stacked bar ────────────────────────────────────
ax2 = fig.add_subplot(gs[1,:2])
ax2.bar(df['Date'],df['cbp_net'].clip(lower=0),color=C['bad'],  alpha=0.65,width=1,label='CBP Accumulation')
ax2.bar(df['Date'],df['hhs_net'].clip(lower=0),
        bottom=df['cbp_net'].clip(lower=0),
        color=C['blue'],alpha=0.65,width=1,label='HHS Accumulation')
ax2.bar(df['Date'],df['cbp_net'].clip(upper=0),color=C['good'],alpha=0.55,width=1,label='CBP Clearing')
ax2.bar(df['Date'],df['hhs_net'].clip(upper=0),
        bottom=df['cbp_net'].clip(upper=0),
        color='#76D7C4',alpha=0.55,width=1,label='HHS Clearing')
ax2.axhline(0,color='black',lw=1.5)
ax2.set(title=f'② Stage Breakdown — PRIMARY BOTTLENECK: {bottleneck}',
        ylabel='Net Change',xlabel='Date')
ax2.legend(fontsize=9,ncol=2)

# ─── CHART 3: Streak duration bar ─────────────────────────────────────────────
ax3 = fig.add_subplot(gs[1,2])
bins={'1–6d': ((streaks>=1)&(streaks<7)).sum(),
      '7–13d':((streaks>=7)&(streaks<14)).sum(),
      '14–29d':((streaks>=14)&(streaks<30)).sum(),
      '30+d': (streaks>=30).sum()}
bc3=[C['blue'],C['warn'],C['bad'],C['bad']]
bars3=ax3.bar(range(4),list(bins.values()),color=bc3,edgecolor='white',lw=1.5)
ax3.set(title='③ Backlog Streak\nDuration Analysis',
        xticks=range(4),ylabel='Number of Streaks')
ax3.set_xticklabels(list(bins.keys()),fontsize=9)
for b,v in zip(bars3,bins.values()):
    if v: ax3.text(b.get_x()+b.get_width()/2,v+0.2,str(v),
                   ha='center',va='bottom',fontsize=11,fontweight='bold')
ax3.text(0.97,0.97,f'Longest: {max_streak}d',
         transform=ax3.transAxes,ha='right',va='top',fontsize=11,
         bbox=dict(boxstyle='round',facecolor=C['lb'],alpha=0.9))

# ─── CHART 4: Total system load area chart ───────────────────────────────────
ax4 = fig.add_subplot(gs[2,:2])
ax4.stackplot(df['Date'],df['cbp_cus'],df['hhs_care'],
              labels=['CBP Custody','HHS Care'],
              colors=[C['bad'],C['blue']],alpha=0.65)
ax4.set(title='④ Total System Load — Active Cases at Each Stage',
        xlabel='Date',ylabel='Active Cases')
ax4.legend(fontsize=10,loc='upper left')
ax4_r=ax4.twinx()
ax4_r.plot(df['Date'],df['sys_load'],lw=2.5,color=C['dark'],
           ls='--',label='Total Load',zorder=5)
ax4_r.set_ylabel('Total Load',color=C['dark'])
ax4_r.legend(fontsize=9,loc='upper right')

# ─── CHART 5: Monthly net-change heatmap ─────────────────────────────────────
ax5 = fig.add_subplot(gs[2,2])
pivot=df.groupby(['year','month'])['net_change'].mean().unstack()
sns.heatmap(pivot,ax=ax5,cmap='RdYlGn_r',center=0,
            annot=True,fmt='.0f',annot_kws={'size':8},
            linewidths=0.5,cbar_kws={'label':'Avg Net Change/Day'})
ax5.set(title='⑤ Monthly Avg Net\nChange Heatmap',
        xlabel='Month',ylabel='Year')
ax5.set_xticklabels(MN,fontsize=8)

plt.savefig('KPI4_Backlog_Accumulation.png',dpi=150,bbox_inches='tight')
plt.show()
print(f"KPI4 done | AvgAcc={avg_acc:+.1f}/day | Bottleneck={bottleneck} | MaxStreak={max_streak}d")

---
## 📊 KPI #5 — Outcome Stability Score
> *Consistency of placements | Formula: CV = σ/μ | Target: CV < 0.3*

**Charts:** ① Shewhart Control Chart  ② Rolling Variability  ③ Quarterly CV  ④ Distribution + KDE  ⑤ Stability Scorecard

In [ ]:
# ═══════════════════════════════════════════════════════════════════
#  KPI 5 ▸ OUTCOME STABILITY SCORE
#  Formula: CV = σ / μ  of Discharge Effectiveness  |  Target: CV < 0.3
#  Story: Are placement outcomes consistent and reliable?
# ═══════════════════════════════════════════════════════════════════

de_mean = df['discharge_eff'].mean()
de_std  = df['discharge_eff'].std()
cv      = de_std/de_mean
ucl     = de_mean+de_std
lcl     = max(0,de_mean-de_std)
ucl2    = de_mean+2*de_std
lcl2    = max(0,de_mean-2*de_std)
in_ctrl = ((df['discharge_eff']>=lcl)&(df['discharge_eff']<=ucl)).sum()
consist = in_ctrl/len(df)*100
drops   = df[df['de_diff']< -0.05]
out_hi  = df[df['discharge_eff']>ucl]
out_lo  = df[df['discharge_eff']<lcl]

if   cv<0.2: rating,rc='★★★★★ HIGHLY STABLE',C['good']
elif cv<0.3: rating,rc='★★★★☆ MODERATELY STABLE',C['blue']
elif cv<0.5: rating,rc='★★★☆☆ UNSTABLE',C['warn']
else:        rating,rc='★☆☆☆☆ HIGHLY ERRATIC',C['bad']

fig = plt.figure(figsize=(22,20))
fig.patch.set_facecolor('white')
fig.text(0.5,0.98,'KPI #5 — Outcome Stability Score',
         ha='center',fontsize=24,fontweight='bold',color=C['dark'])
fig.text(0.5,0.955,
         f'Consistency of placements  |  Formula: CV = σ/μ  |  Target CV < 0.3'
         f'  |  CV = {cv:.3f}  |  {consist:.0f}% days in control  |  {rating}',
         ha='center',fontsize=12,color=rc)

gs = gridspec.GridSpec(3,3,figure=fig,top=0.94,bottom=0.05,
                       hspace=0.50,wspace=0.38)

# ─── CHART 1: Control chart (Shewhart) ───────────────────────────────────────
ax1 = fig.add_subplot(gs[0,:])
ax1.fill_between(df['Date'],lcl2,ucl2,color=C['lg'],alpha=0.20,label='±2σ Zone')
ax1.fill_between(df['Date'],lcl, ucl, color=C['lg'],alpha=0.35,label=f'Normal Range ±1σ ({consist:.0f}% days)')
ax1.plot(df['Date'],df['discharge_eff'],lw=1.2,color=C['purple'],alpha=0.45,
         marker='o',markersize=2,label='Daily Rate')
ax1.axhline(de_mean,color=C['good'],lw=3.0,ls='-', label=f'Mean {de_mean:.2%}')
ax1.axhline(ucl,    color=C['warn'],lw=1.8,ls='--',label=f'UCL +1σ {ucl:.2%}')
ax1.axhline(lcl,    color=C['warn'],lw=1.8,ls='--',label=f'LCL -1σ {lcl:.2%}')
if len(out_hi): ax1.scatter(out_hi['Date'],out_hi['discharge_eff'],
                             color=C['bad'],s=100,zorder=5,marker='^',
                             edgecolors='black',lw=1.5,label=f'Above UCL ({len(out_hi)})')
if len(out_lo): ax1.scatter(out_lo['Date'],out_lo['discharge_eff'],
                             color=C['bad'],s=100,zorder=5,marker='v',
                             edgecolors='black',lw=1.5,label=f'Below LCL ({len(out_lo)})')
if len(drops):  ax1.scatter(drops['Date'],drops['discharge_eff'],
                             color='darkred',s=150,zorder=6,marker='X',
                             edgecolors='yellow',lw=2,label=f'Sudden Drops ({len(drops)})')
ax1.set(title='① Shewhart Control Chart — Discharge Effectiveness Stability',
        ylabel='Discharge Effectiveness',xlabel='Date')
ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda y,_:f'{y:.1%}'))
ax1.legend(ncol=3,fontsize=9,loc='upper left',framealpha=0.9)
ax1.text(0.01,0.96,f'CV={cv:.3f} | {consist:.0f}% in control | Rating: {rating}',
         transform=ax1.transAxes,fontsize=10,va='top',
         bbox=dict(boxstyle='round',facecolor=C['lw'],alpha=0.9))

# ─── CHART 2: Rolling std dev variability ────────────────────────────────────
ax2 = fig.add_subplot(gs[1,:2])
ax2.plot(df['Date'],df['dc_roll_std'],lw=2.5,color=C['purple'],label='30-Day Rolling σ')
ax2.fill_between(df['Date'],0,df['dc_roll_std'],alpha=0.22,color=C['purple'])
ax2.axhline(de_std,color='blue',lw=2,ls='--',label=f'Overall σ = {de_std:.3f}')
stable_thresh=0.02
ax2.axhline(stable_thresh,color=C['good'],lw=1.8,ls=':',label=f'Stable Threshold {stable_thresh}')
ax2.fill_between(df['Date'],0,stable_thresh,alpha=0.15,color=C['good'])
ax2.set(title='② 30-Day Rolling Variability — Periods of High vs Low Outcome Consistency',
        xlabel='Date',ylabel='Rolling Std Dev')
ax2.legend(fontsize=10)

# ─── CHART 3: Quarterly CV comparison ────────────────────────────────────────
ax3 = fig.add_subplot(gs[1,2])
q_cv=df.groupby('quarter').apply(
    lambda x: x['discharge_eff'].std()/x['discharge_eff'].mean()
).dropna()
q_labs=q_cv.index.tolist()
bc3=[C['good'] if v<0.3 else (C['warn'] if v<0.5 else C['bad']) for v in q_cv.values]
bars3=ax3.bar(range(len(q_cv)),q_cv.values,color=bc3,edgecolor='white',lw=1.5)
ax3.axhline(0.3,color=C['warn'],lw=2,ls='--',label='Threshold 0.3')
ax3.axhline(cv,color='blue',lw=2,ls='--',alpha=0.7,label=f'Overall CV {cv:.3f}')
ax3.set(title='③ Quarterly CV\n(Stability per Period)',
        xticks=range(len(q_cv)),ylabel='CV (lower=stable)')
ax3.set_xticklabels(q_labs,rotation=45,ha='right',fontsize=7.5)
ax3.legend(fontsize=8)
for b,v in zip(bars3,q_cv.values):
    ax3.text(b.get_x()+b.get_width()/2,v+0.005,f'{v:.2f}',
             ha='center',va='bottom',fontsize=8,fontweight='bold')

# ─── CHART 4: Distribution + KDE ─────────────────────────────────────────────
ax4 = fig.add_subplot(gs[2,:2])
sns.histplot(df['discharge_eff'].dropna(),bins=70,ax=ax4,
             color=C['purple'],alpha=0.60,edgecolor='white',stat='count')
ax4_k=ax4.twinx()
sns.kdeplot(df['discharge_eff'].dropna(),ax=ax4_k,
            color=C['dark'],lw=2.5,label='KDE')
ax4_k.set_ylabel('Density')
ax4.axvline(de_mean,color=C['good'], lw=2.5,ls='-', label=f'Mean {de_mean:.2%}')
ax4.axvline(ucl,    color=C['warn'], lw=2.0,ls='--',label=f'UCL {ucl:.2%}')
ax4.axvline(lcl,    color=C['warn'], lw=2.0,ls='--',label=f'LCL {lcl:.2%}')
ax4.axvline(0.05,   color=C['bad'],  lw=2.0,ls=':' ,label='Target 5%')
ax4.set(title='④ Distribution of Discharge Effectiveness — Shape Reveals Stability Pattern',
        xlabel='Discharge Effectiveness',ylabel='Number of Days')
ax4.xaxis.set_major_formatter(plt.FuncFormatter(lambda x,_:f'{x:.1%}'))
l1,lb1=ax4.get_legend_handles_labels()
l2,lb2=ax4_k.get_legend_handles_labels()
ax4.legend(l1+l2,lb1+lb2,fontsize=9)

# ─── CHART 5: Stability scorecard ─────────────────────────────────────────────
ax5 = fig.add_subplot(gs[2,2])
ax5.axis('off')
sc=[
    ('Mean (μ)',f'{de_mean:.3%}'),
    ('Std Dev (σ)',f'{de_std:.4f}'),
    ('CV Score',f'{cv:.3f}'),
    ('UCL (+1σ)',f'{ucl:.3%}'),
    ('LCL (-1σ)',f'{lcl:.3%}'),
    ('In Control',f'{in_ctrl} days ({consist:.0f}%)'),
    ('Above UCL',f'{len(out_hi)} days'),
    ('Below LCL',f'{len(out_lo)} days'),
    ('Sudden Drops',f'{len(drops)} events'),
    ('Stability',rating),
    ('Status','✓ STABLE' if cv<0.3 else '✗ UNSTABLE'),
]
ax5.text(0.5,1.02,'KPI #5 Scorecard',ha='center',va='top',
         fontsize=13,fontweight='bold',transform=ax5.transAxes)
yp=0.89
for lbl,val in sc:
    stable_col=C['good'] if ('✓' in val or cv<0.3) else (C['bad'] if '✗' in val else C['dark'])
    ax5.text(0.03,yp,lbl+':',fontsize=9.5,va='top',
             transform=ax5.transAxes,color='gray')
    ax5.text(0.97,yp,val,fontsize=9.5,va='top',ha='right',
             transform=ax5.transAxes,fontweight='bold',
             color=C['good'] if '✓' in val else (C['bad'] if '✗' in val else C['dark']))
    yp-=0.086

facecolor=C['lg'] if cv<0.3 else C['lb']
ax5.add_patch(plt.Rectangle((0,0),1,1,transform=ax5.transAxes,
                              facecolor=facecolor,alpha=0.25,zorder=0))

plt.savefig('KPI5_Outcome_Stability.png',dpi=150,bbox_inches='tight')
plt.show()
print(f"KPI5 done | CV={cv:.3f} | Consistency={consist:.0f}% | {rating}")

---
## 🎯 Executive Summary Dashboard — All 5 KPIs

In [ ]:
# ═══════════════════════════════════════════════════════════════════
#  EXECUTIVE SUMMARY DASHBOARD — All 5 KPIs Side-by-Side
# ═══════════════════════════════════════════════════════════════════

fig,axes = plt.subplots(5,1,figsize=(18,22))
fig.patch.set_facecolor('white')
fig.suptitle('UAC Care Pipeline — Executive KPI Dashboard\n5 KPIs at a Glance',
             fontsize=20,fontweight='bold',color=C['dark'],y=0.99)

kpi_data=[
    ('KPI #1 Transfer Efficiency','transfer_eff','transfer_eff_30',
     0.8,'Policy Target 80%',C['blue'],
     f"Mean {df['transfer_eff'].mean():.1%}  |  Avg CBP stay {1/df['transfer_eff'].mean():.1f}d"),
    ('KPI #2 Discharge Effectiveness','discharge_eff','discharge_eff_30',
     0.05,'Min Target 5%',C['purple'],
     f"Mean {df['discharge_eff'].mean():.2%}  |  Avg HHS stay {1/df['discharge_eff'].mean():.0f}d"),
    ('KPI #3 Pipeline Throughput','throughput','throughput_30',
     1.0,'Balanced 100%',C['warn'],
     f"Overall {(df['hhs_dc'].sum()/df['cbp_in'].sum()):.1%}  |  Gap {df['cbp_in'].sum()-df['hhs_dc'].sum():,.0f}"),
    ('KPI #4 Backlog (Net Change)','net_change',None,
     0,'Zero Accumulation',C['bad'],
     f"Avg {df['net_change'].mean():+.1f}/day  |  Final backlog {df['cum_backlog'].iloc[-1]:+,.0f}"),
    ('KPI #5 Outcome Stability (DC Eff)','discharge_eff','discharge_eff_30',
     None,None,C['good'],
     f"CV={df['discharge_eff'].std()/df['discharge_eff'].mean():.3f}  |  Consist. {((df['discharge_eff']>=df['discharge_eff'].mean()-df['discharge_eff'].std())&(df['discharge_eff']<=df['discharge_eff'].mean()+df['discharge_eff'].std())).sum()/len(df)*100:.0f}%"),
]

fmt = plt.FuncFormatter(lambda y,_: f'{y:.0%}')

for ax,(title,col,smooth,target,tlabel,color,annotation) in zip(axes,kpi_data):
    if col=='net_change':
        bc=[C['bad'] if v>0 else C['good'] for v in df[col]]
        ax.bar(df['Date'],df[col],color=bc,alpha=0.55,width=1)
        ax.axhline(0,color='black',lw=1.5)
    else:
        ax.plot(df['Date'],df[col],lw=1.0,color=color,alpha=0.35)
        if smooth: ax.plot(df['Date'],df[smooth],lw=2.5,color=color)
        if col=='KPI #5':
            m=df[col].mean(); s=df[col].std()
            ax.fill_between(df['Date'],m-s,m+s,alpha=0.2,color=C['good'])
    if target is not None:
        ax.axhline(target,color='red',lw=2.0,ls='--',label=tlabel,alpha=0.9)
        ax.legend(fontsize=9,loc='lower right')
    ax.set_facecolor(C['bg'])
    ax.spines[['top','right']].set_visible(False)
    ax.set_title(title,fontsize=13,fontweight='bold',color=C['dark'],loc='left')
    ax.text(0.99,0.96,annotation,transform=ax.transAxes,
            ha='right',va='top',fontsize=10,
            bbox=dict(boxstyle='round',facecolor='white',alpha=0.85))
    for sp in ['left','bottom']: ax.spines[sp].set_color('#BDC3C7')
    ax.grid(True,alpha=0.3,ls='--')
    ax.tick_params(labelsize=9)
    if col not in ['net_change']:
        if col in ['transfer_eff','discharge_eff','throughput']:
            ax.yaxis.set_major_formatter(fmt)
    ax.set_xlabel('Date',fontsize=9)

plt.tight_layout(rect=[0,0,1,0.98])
plt.savefig('KPI_Executive_Dashboard.png',dpi=150,bbox_inches='tight')
plt.show()
print("Executive dashboard saved!")